In [181]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

In [71]:
sales_train = pd.read_csv('sales_train.csv')
shops = pd.read_csv('shops.csv')
items = pd.read_csv('items.csv')
item_categories = pd.read_csv('item_categories.csv')

In [72]:
sales_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [73]:
#Merge shops and sales_train
t1= sales_train.merge(shops, on='shop_id',how='left')

In [74]:
#Merge it again with items
t1 = t1.merge(items, on='item_id',how='left')

In [75]:
#Merge it again with item categories
t1=t1.merge(item_categories, on='item_category_id',how='left')

In [76]:
t1.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,shop_name,item_name,item_category_id,item_category_name
0,02.01.2013,0,59,22154,999.00,1.0,"Ярославль ТЦ ""Альтаир""",ЯВЛЕНИЕ 2012 (BD),37,Кино - Blu-Ray
1,03.01.2013,0,25,2552,899.00,1.0,"Москва ТРК ""Атриум""",DEEP PURPLE The House Of Blue Light LP,58,Музыка - Винил
2,05.01.2013,0,25,2552,899.00,-1.0,"Москва ТРК ""Атриум""",DEEP PURPLE The House Of Blue Light LP,58,Музыка - Винил
3,06.01.2013,0,25,2554,1709.05,1.0,"Москва ТРК ""Атриум""",DEEP PURPLE Who Do You Think We Are LP,58,Музыка - Винил
4,15.01.2013,0,25,2555,1099.00,1.0,"Москва ТРК ""Атриум""",DEEP PURPLE 30 Very Best Of 2CD (Фирм.),56,Музыка - CD фирменного производства


In [96]:
t1.to_csv('train_full.csv')

In [79]:
#Perform same merge operations with test
test=pd.read_csv('test.csv', index_col='ID')
t2 = test.merge(shops, on='shop_id',how='left')
t2 = t2.merge(items, on='item_id',how='left')
t2=t2.merge(item_categories, on='item_category_id',how='left')

In [95]:
t2.to_csv('test_full.csv')

In [94]:
t2.head()

,shop_id,item_id,shop_name,item_name,item_category_id,item_category_name,date_block_num
0,5,5037,"Вологда ТРЦ ""Мармелад""","NHL 15 [PS3, русские субтитры]",19,Игры - PS3,34
1,5,5320,"Вологда ТРЦ ""Мармелад""",ONE DIRECTION Made In The A.M.,55,Музыка - CD локального производства,34
2,5,5233,"Вологда ТРЦ ""Мармелад""","Need for Speed Rivals (Essentials) [PS3, русск...",19,Игры - PS3,34
3,5,5232,"Вологда ТРЦ ""Мармелад""","Need for Speed Rivals (Classics) [Xbox 360, ру...",23,Игры - XBOX 360,34
4,5,5268,"Вологда ТРЦ ""Мармелад""","Need for Speed [PS4, русская версия]",20,Игры - PS4,34


In [93]:
#Fill test date_block_num with next month indices
t2['date_block_num']= [t1['date_block_num'].iloc[-1]+1]*t2.shape[0]

In [100]:
#Check for NA
t1na = pd.isna(t1)
[set(t1na.iloc[:,i]) for i in range(t1na.shape[1])]

[{False},
 {False},
 {False},
 {False},
 {False},
 {False},
 {False},
 {False},
 {False},
 {False}]

In [128]:
item_cnt_day_vc = t1['item_cnt_day'].value_counts()


In [143]:
items_cnt_sorted = sorted(item_cnt_day_vc.items())

In [126]:
min(t1['item_cnt_day'])

-22.0

In [124]:
item_cnt_day_1p = np.log1p(2+t1['item_cnt_day'])

In [155]:
#Check if new shop_ids are present in test that are not in train
t1_shop_id = set(t1['shop_id'])
t2_shop_id = set(t2['shop_id'])
missing_id=[]
for t in t2_item_id:
    if t not in t1_item_id:
        missing_id.append(t)
print('is missing?',len(missing_id)!=0)

In [158]:
#Check if new item_ids are present in test that are not in train
t1_item_id = set(t1['item_id'])
t2_item_id = set(t2['item_id'])
missing_id=[]
for t in t2_item_id:
    if t not in t1_item_id:
        missing_id.append(t)
print('is missing?',len(missing_id)!=0)

In [188]:
#To fill the missing item_id in test, group item_category and aggregate with the mode of item_id
def mymode(arr):
    return stats.mode(arr)[0][0]

item_category_id_mode = t1.groupby('item_category_id')['item_id'].agg(mymode)
item_category_id_mode = pd.DataFrame(item_category_id_mode)
item_cat=  items[['item_id','item_category_id']].set_index('item_id')
item_cat_missing = item_cat.loc[missing_id,:]
item_cat_missing = item_cat_missing.reset_index()
item_category_id_mode=item_category_id_mode.reset_index()
item_item = item_cat_missing.merge(item_category_id_mode,on='item_category_id',how='left').drop('item_category_id',1)
item_item=item_item.set_index('item_id_x')
item_item = item_item.to_dict()['item_id_y']

test_item_id=[]
for a in t2.item_id:
    if a in item_item:
        test_item_id.append(item_item[a])
    else:
        test_item_id.append(a)
        
t2['item_id']=test_item_id        

In [265]:
t2.to_csv('test_full.csv')

In [266]:
#Feature Engineering
#TODO impute item_price

In [ ]:
features_cols=['date_block_num', 'shop_id', 'item_id', 'item_price', 'shop_name', 'item_name', 'item_category_id',
       'item_category_name']
features=pd.concat([t1[features_cols],t2[features_cols]], axis=0)